# Feature Selection

### Libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sqlite3
from sklearn import preprocessing

from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import ConfusionMatrix
from yellowbrick.classifier import ClassificationReport
from yellowbrick.features.importances import FeatureImportances

### Import Data

In [ ]:
conn = sqlite3.connect('../data/crime_census_weather_tod.db')        
df = pd.read_sql_query("select * from all_crimes", conn)
conn.close()

df.columns

### Encode/Standardize Features

Encode Blockgroup, which includes an embedded Tract.

In [ ]:
le = preprocessing.LabelEncoder()
bg_fit = le.fit(df['BLOCK_GROUP'])

In [ ]:
df['bg_cat'] = bg_fit.transform(df['BLOCK_GROUP'])

print(df.columns, len(df.columns))

Drop missing values.

In [ ]:
df_m = df.dropna(how='any')

print(len(df),len(df_m), len(df)-len(df_m))

Standardize

In [ ]:
feature_std = [
    'year', 'month', 'day', 'tod_num',
    'cloud_cover', 'dew_point', 'humidity', 'percip_intensity',
    'percip_probability', 'pressure', 'temperature', 'uv_index',
    'visibility', 'wind_bearing', 'wind_gust', 'wind_speed',
    'PerCapitaIncome', 'MedianHouseholdInc', 'MedianAge', 'HousingUnits',
    'weekday', 'bg_cat'
]

scaler = preprocessing.StandardScaler()
scaler_df= pd.DataFrame(scaler.fit_transform(df_m[feature_std]), columns=feature_std)

print(len(feature_std))

### Feature Selection

In [ ]:
feature_cols = [
    'year', 'month', 'day', 'tod_num',
    'cloud_cover', 'dew_point', 'humidity', 'percip_intensity',
    'percip_probability', 'pressure', 'temperature', 'uv_index',
    'visibility', 'wind_bearing', 'wind_gust', 'wind_speed',
    'PerCapitaIncome', 'MedianHouseholdInc', 'MedianAge',
    'weekday', 'bg_cat'
]

len(feature_cols)

In [ ]:
features = scaler_df[feature_cols]
target = df_m['crime_rate_cat']

#### Feature ranking

Feature ranking with recursive feature elimination and cross-validated selection for the best number of features.

Random Forest Classifier

In [ ]:
rmf = RandomForestClassifier()
rfecv = RFECV(estimator=rmf,
              step=1,
              cv=StratifiedKFold(2), #Set to 12 for production.
              scoring='accuracy',
              verbose=10,
              n_jobs=-1)

In [ ]:
rfecv.fit(features_scaled, target)
print(rfecv.n_features_)

In [ ]:
plt.figure(figsize=(15, 10))
plt.xlabel("Number of Features Selected")
plt.ylabel("Cross Validation Score")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
for index in np.where(rfecv.support_)[0]:
    print(features_scaled.columns[index])

#### Feature Importances

Random Forest Classifier

In [ ]:
# Create a new matplotlib figure
fig = plt.figure()
ax = fig.add_subplot()

viz = FeatureImportances(RandomForestClassifier(), ax=ax)
viz.fit(features, target)
viz.poof()

### Modeling

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2)

In [ ]:
classes=list(target.unique())
classes

#### ROCAUC

In [ ]:
visualizer = ROCAUC(RandomForestClassifier(), classes=classes)

visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
g = visualizer.poof()   

#### Classification Reports

In [ ]:
models = [
    RandomForestClassifier(),
    BaggingClassifier(),
    ExtraTreesClassifier(),
    GradientBoostingClassifier(),
    KNeighborsClassifier(),
    DecisionTreeClassifier()
]

In [ ]:
for model in models:
    cr = ClassificationReport(model, classes=classes)
    cr.fit(X_train, y_train)
    cr.score(X_test, y_test)
    cr.poof()

#### Confusion Matricies

In [ ]:
models = [
    RandomForestClassifier(),
    BaggingClassifier(),
    ExtraTreesClassifier(),
    GradientBoostingClassifier(),
    KNeighborsClassifier(),
    DecisionTreeClassifier()
]

In [ ]:
for model in models:
    cm = ConfusionMatrix(model, classes=classes)
    cm.fit(X_train, y_train)
    cm.score(X_test, y_test)
    cm.poof()